In [5]:
import numpy as np
import matplotlib.pyplot as plt
import folium
from ipywidgets import interact
import pandas as pd
from shapely import intersection
from shapely.geometry import MultiPoint, Polygon, Point
from shapely import voronoi_polygons

def random_color():
    color = np.random.randint(16, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    return '#' + ''.join(color).upper()

In [2]:
data = pd.read_csv("bike_share_data/cleaned_data.csv")

### Voronoi digram plotter

In [3]:
def generate_points(n):
    return data[['LATITUDE','LONGITUDE']].to_numpy()
    
def plot_voronoi(points):
    points2 = MultiPoint(points)
    envelope = Polygon([(38.7, -76.8), (38.7,-77.4), (39.2,-77.4), (39.2,-76.8)])
    # Compute the Voronoi diagram
    voronoi_diagram = voronoi_polygons(points2)
    # vor_polygons_in_country = shapely.intersection(vor_polygons[0], envelope)
    vor_polygons = []
    for geom in voronoi_diagram.geoms:
        xx, yy = geom.exterior.coords.xy
        coord = list(zip(xx, yy))
        # inter = intersection(Polygon(coord), envelope)
        inter = intersection(geom, envelope)
        xx, yy = inter.exterior.coords.xy
        coord_clipped = list(zip(xx, yy))
        
        vor_polygons.append(coord_clipped)
    
    
    # Create Folium map
    map_center = points.mean(axis=0)
    m = folium.Map(location=map_center, zoom_start=10)
    
    # Add Voronoi edges to map
    for poly in vor_polygons:
        folium.Polygon(
            locations=poly,
            color="blue",
            weight=4,
            fill_color=random_color(),
            fill_opacity=0.5,
            fill=True,
            popup="add station name",
            tooltip="Click me!").add_to(m)
    
    for point in points:
        folium.CircleMarker(location=point,
                        radius=2,
                        weight=2).add_to(m)
    
    # Display map
    display(m)
# @interact(locations_to_remove=(0,10))
@interact(Add=(0, 10), Remove=(0,10))
def adjust_points(Add, Remove):
    points = generate_points(Add)
    plot_voronoi(points)


interactive(children=(IntSlider(value=5, description='Add', max=10), IntSlider(value=5, description='Remove', …

### Slower plotting but has bike share location names in region

In [6]:
def generate_points(n):
    return data[['LATITUDE','LONGITUDE']].to_numpy()
    
def plot_voronoi(points):
    # Define your input points (vertices)
    # points = data[['LATITUDE','LONGITUDE']].to_numpy()
    points2 = MultiPoint(points)
    envelope = Polygon([(38.7, -76.8), (38.7,-77.4), (39.2,-77.4), (39.2,-76.8)])
    # Compute the Voronoi diagram
    voronoi_diagram = voronoi_polygons(points2)
    # vor_polygons_in_country = shapely.intersection(vor_polygons[0], envelope)
    vor_polygons = []
    for geom in voronoi_diagram.geoms:
        xx, yy = geom.exterior.coords.xy
        coord = list(zip(xx, yy))
        # inter = intersection(Polygon(coord), envelope)
        inter = intersection(geom, envelope)
        xx, yy = inter.exterior.coords.xy
        coord_clipped = list(zip(xx, yy))
        
        vor_polygons.append(coord_clipped)
    
    
    
    # Create Folium map
    map_center = points.mean(axis=0)
    m = folium.Map(location=map_center, zoom_start=10)
    
    # Add Voronoi edges to map
    for poly in vor_polygons:
        for index, row in data.iterrows():
             if Polygon(poly).contains(Point(row['LATITUDE'],row['LONGITUDE'])):
                 folium.Polygon(
                    locations=poly,
                    color="blue",
                    weight=4,
                    fill_color=random_color(),
                    fill_opacity=0.5,
                    fill=True,
                    popup=f"{row['station_name']}",
                    ).add_to(m)
    
                 folium.CircleMarker(location=[row['LATITUDE'],row['LONGITUDE']],
                        radius=2,
                        weight=2).add_to(m)
                 break
                 
    # Display map
    display(m)

# @interact(locations_to_remove=(0,10))
@interact(Add=(0, 10), Remove=(0,10))
def adjust_points(Add, Remove):
    points = generate_points(Add)
    plot_voronoi(points)

interactive(children=(IntSlider(value=5, description='Add', max=10), IntSlider(value=5, description='Remove', …